# 1. LIBRARIES

In [47]:
import pandas as pd
import os
import re  # Importamos para manejar expresiones regulares
import pandas as pd

# 2. LOCAL VARIABLES

In [48]:
#Variables globales
#Nombre del dataframe final
Regimen = "NCEPSC25"
#Regimen = "NSEPS025"
nombre_merged = "neg_all_2024"
# Definir la ruta donde se guardará el archivo Excel basado en la variable nombre_merged
output_name = f"{nombre_merged}.xlsx"
output_path = os.path.join(r"C:/Users/crist/OneDrive - Corporación Universitaria Remington UNIREMINGTON/Estudio/Ciencia de datos/Lumethik/eps-data-solutions/data/raw/Procesos BDUA/Contributivo/Negados/Files since 2018 until 2024/", output_name)
#output_path = os.path.join(r"C:\Users\carlo\Downloads", output_name)


# 3. READING OF FOLDER WITH DATA NEGADOS OR VALIDADOS

In [ ]:
def obtener_ruta_carpeta():
    folder_path = input('Por favor, ingresa la ruta de la carpeta (incluyendo las comillas si están presentes): ')
    folder_path = folder_path.strip('"').replace("\\", "/")
    if not os.path.exists(folder_path):
        print(f"La ruta {folder_path} no existe.")
        return None
    return folder_path

def cargar_archivos_a_dataframe(folder_path, extensiones=[".NEG", ".VAL"], sep=",", encoding="latin-1"):
    if not os.path.exists(folder_path):
        print(f"La ruta {folder_path} no es válida.")
        return {}
    
    dataframes = {}
    for ext in extensiones:
        for filename in os.listdir(folder_path):
            if filename.endswith(ext):
                file_path = os.path.join(folder_path, filename)
                try:
                    df = pd.read_csv(file_path, sep=sep, encoding=encoding, header=None)
                    df.columns = [f"col_{j+1}" for j in range(df.shape[1])]
                    
                    nombre_base = os.path.splitext(filename)[0]  # Nombre sin extensión
                    nombre_base = nombre_base.replace(Regimen, "")  # Eliminar "NSEPS025" del nombre
                    clave = f"df_{nombre_base}"  # Clave personalizada
                    dataframes[clave] = df
                    print(f"Archivo {filename} cargado como {clave}")
                except Exception as e:
                    print(f"No se pudo leer el archivo {filename}: {e}")
    
    total_df = len(dataframes)
    print(f"\nSe crearon {total_df} DataFrames en total.")
    
    if not dataframes:
        print("No se cargaron archivos en la carpeta especificada.")
    return dataframes

def agregar_nombre_a_dataframes(dataframes):
    """
    Agrega una columna adicional a cada DataFrame con su nombre (clave del diccionario).
    """
    for name, df in dataframes.items():
        df["nombre_dataframe"] = name  # Agrega una columna con el nombre del DataFrame
    print("Se agregó la columna 'nombre_dataframe' a todos los DataFrames.")

def mostrar_primeras_filas(dataframes):
    for name, df in dataframes.items():
        print(f"Primeras filas de {name}:")
        print(df.head())
        print("\n---\n")
        # Puedes quitar el "break" si deseas mostrar todos los DataFrames
        break

# Ejecución del script
ruta = obtener_ruta_carpeta()
if ruta:
    # Incluye las extensiones .NEG y .VAL
    dataframes = cargar_archivos_a_dataframe(ruta, extensiones=[".NEG", ".VAL"])
    if dataframes:
        # Agregar el nombre del DataFrame como columna
        agregar_nombre_a_dataframes(dataframes)
        # Mostrar las primeras filas de los DataFrames
        mostrar_primeras_filas(dataframes)


Revisando las dimenciones de los dataframes

In [50]:
# Si df_1 está en el diccionario 'dataframes', lo puedes acceder así:

#dataframes['df_22112019'].head(3)


## 3.1 Searching the shape of all dataframes

In [ ]:
print('Name  Row  Cols')

# Lista para almacenar la cantidad de filas
row_counts = []

# Recorremos los DataFrames en el diccionario
for name, df in dataframes.items():
    rows, cols = df.shape
    row_counts.append(rows)  # Añadimos la cantidad de filas a la lista
    print(f"{name}: {rows}  {cols}")

# Calculamos el total de filas
total_rows = sum(row_counts)
print(f"\nTotal de registros en todos los DataFrames: {total_rows}")


# 4. Creating the original Merged

In [ ]:
# Definir el número de columnas deseadas
num_columnas = 22

# Asegurar que todos los DataFrames tengan exactamente el mismo número de columnas
dataframes_alineados = {name: df.iloc[:, :num_columnas] for name, df in dataframes.items()}

# Definir un nombre fijo para el DataFrame combinado
#nombre_merged = "merged_meses"

# Combinar todos los DataFrames alineados, sin crear nuevas columnas
globals()[nombre_merged] = pd.concat(dataframes_alineados.values(), axis=0, ignore_index=True)

# Mostrar el shape del DataFrame combinado
print(f"Shape del DataFrame '{nombre_merged}': {globals()[nombre_merged].shape}")

# Verificar las primeras filas
print(f"Primeras 5 filas del DataFrame '{nombre_merged}':")
print(globals()[nombre_merged].head())


## 4.1 Checking the Merged's Columns

In [ ]:
print(globals()[nombre_merged].columns)

## 4.2 Cleaning the Merged's colums

In [ ]:
# Selecciona las columnas que deseas mantener
globals()[nombre_merged] = globals()[nombre_merged][['col_2', 'col_3','col_4','col_9', 'col_10', 'col_11', 'col_12', 'col_13','nombre_dataframe','col_21']]
#union_2019 = union_2019[['col_2', 'col_3','col_8','col_9', 'col_10', 'col_11', 'col_12', 'col_13', 'col_21']]
# Verifica el resultado
print(globals()[nombre_merged].head())

## 4.3 Renaming column names

In [ ]:
# Obtiene la cantidad de columnas en el DataFrame
num_columns = globals()[nombre_merged].shape[1]

# Genera nuevos nombres de columnas: 'colum1', 'colum2', ..., 'columnN'
new_columns = [f'colum{i+1}' for i in range(num_columns)]

# Asigna los nuevos nombres de columnas al DataFrame
globals()[nombre_merged].columns = new_columns

# Verifica el resultado
print(globals()[nombre_merged].head())


In [ ]:
globals()[nombre_merged].dtypes

## 4.4 Looking for the column with df_ and converting to date

In [ ]:
#import re  # Importamos para manejar expresiones regulares

# Procesamos directamente el DataFrame en globals()
for column in globals()[nombre_merged].columns:
    # Modificamos las celdas de cada columna según el patrón
    globals()[nombre_merged][column] = globals()[nombre_merged][column].astype(str).apply(
        lambda x: re.sub(r'^df_(\d{2})(\d{2})(\d{4})$', r'\1/\2/\3', x) 
        if 'df_' in x else x
    )

# Confirmamos que la transformación se realizó
print(f"Transformación realizada directamente en el DataFrame: {nombre_merged}")
print(globals()[nombre_merged].head())


## 4.5 Change type of columns

In [ ]:
# Recorremos cada columna del DataFrame
for column in globals()[nombre_merged].columns:
    print(f"Procesando columna: {column}")  # Mostrar la columna actual
    
    if column == "colum3":
        # Convertir la columna 'colum3' directamente a string
        print(f"Convirtiendo la columna '{column}' a string")
        globals()[nombre_merged][column] = globals()[nombre_merged][column].astype(str)
        continue  # Pasar a la siguiente columna después de procesar 'colum3'
    
    # Convertir a entero si todos los valores son numéricos
    if globals()[nombre_merged][column].apply(lambda x: str(x).isdigit()).all():
        print(f"Intentando convertir la columna '{column}' a entero")
        globals()[nombre_merged][column] = globals()[nombre_merged][column].astype(int)
    
    # Convertir a string con formato DD/MM/YYYY si los valores están en formato de fecha
    else:
        try:
            print(f"Intentando convertir la columna '{column}' a formato de fecha")
            # Convertir a datetime primero
            globals()[nombre_merged][column] = pd.to_datetime(
                globals()[nombre_merged][column], format='%d/%m/%Y', errors='raise'
            )
            # Convertir a string con formato DD/MM/YYYY
            globals()[nombre_merged][column] = globals()[nombre_merged][column].dt.strftime('%d/%m/%Y')
        except Exception as e:
            print(f"No se pudo convertir la columna '{column}' a fecha: {e}")
            pass  # Si no es fecha, se deja igual

print(f"Conversión de tipos completada en el DataFrame: {nombre_merged}")


In [ ]:
globals()[nombre_merged].dtypes

In [ ]:
globals()[nombre_merged].head(2)

## 4.6 Renaming names Columns

In [61]:
# Renombrar las columnas
globals()[nombre_merged].columns = ['cod_reg', 'tip_doc', 'doc', 'fech_nac', 'dep', 'mun', 'nov', 'fech_nov', 'fecha_rep', 'observs']


In [ ]:
globals()[nombre_merged].shape

In [ ]:
globals()[nombre_merged].head(2)

In [ ]:
globals()[nombre_merged].dtypes

# 4.7 Counting Glosses

In [65]:
# Definir una función para contar el número de glosas en un registro
def count_glosas(observs):
    # Asegúrate de trabajar con cadenas
    if isinstance(observs, str):
        return len(re.findall(r'GN\d{4}', observs))
    # Si no es una cadena, devolver 0
    return 0

# Aplicar la función a la columna 'observs' y crear la nueva columna 'No_Glosas'
globals()[nombre_merged]["No_Glosas"] = globals()[nombre_merged]["observs"].apply(count_glosas)

## 4.7.1 Divide las glosas que estan concatenadas en 1 solo registro

In [ ]:
# 1. Crear nueva columna separando por punto y coma
globals()[nombre_merged]['observaciones_split'] = globals()[nombre_merged]['observs'].str.split(';')

# 2. Usar explode para crear una fila por cada elemento
df_resultado = globals()[nombre_merged].explode('observaciones_split')

# 3. Resetear el índice
df_resultado = df_resultado.reset_index(drop=True)

# 4. Guardar el resultado
globals()[nombre_merged] = df_resultado

# Mostrar resultado
print("\nPrimeras 10 filas del DataFrame procesado:")
print(df_resultado.head(10))

# Mostrar conteo de filas
print(f"\nTotal de filas antes de separar: {len(globals()[nombre_merged])}")
print(f"Total de filas después de separar: {len(df_resultado)}")

## 4.7.2 Creating new files for every glosa per User

In [ ]:
import re

# 1. Crear la nueva columna para las glosas
globals()[nombre_merged]['obs_glos'] = ''

# 2. Función para separar GN y glosa
def separar_gn_glosa(texto):
    if isinstance(texto, str):
        # Buscar específicamente GN seguido de exactamente 4 dígitos
        match = re.match(r'(GN\d{4})(.*)', texto.strip())
        if match:
            return match.group(1), match.group(2).strip()
    return texto, ''

# 3. Aplicar la separación
for idx, row in globals()[nombre_merged].iterrows():
    gn, glosa = separar_gn_glosa(row['observaciones_split'])
    globals()[nombre_merged].at[idx, 'observaciones_split'] = gn
    globals()[nombre_merged].at[idx, 'obs_glos'] = glosa

# 4. Mostrar el resultado
print("\nPrimeras 10 filas después de separar GN y glosas:")
print(globals()[nombre_merged][['observaciones_split', 'obs_glos']].head(10))

# 5. Mostrar algunos ejemplos de las separaciones realizadas
print("\nEjemplos de separaciones realizadas:")
muestra = globals()[nombre_merged][['observaciones_split', 'obs_glos']].sample(5)
print(muestra)

## 4.7.2 Deleting Nan dates in observaciones_split column

In [ ]:
# Eliminar valores vacíos y espacios en blanco
df_resultado = globals()[nombre_merged]
df_resultado = df_resultado[df_resultado['observaciones_split'].notna()]  # Elimina NaN
df_resultado = df_resultado[df_resultado['observaciones_split'].str.strip() != '']  # Elimina strings vacíos o solo espacios

# Guardar el resultado limpio
globals()[nombre_merged] = df_resultado

# Mostrar resultado
print("\nPrimeras 5 filas después de limpiar:")
print(df_resultado.head())

print(f"\nTotal de filas después de limpiar: {len(df_resultado)}")

# 5. DOWNLADING FENITIVE FILE

In [ ]:
# Guardar el DataFrame como un archivo Excel
globals()[nombre_merged].to_excel(output_path, index=False)

print(f"El archivo se ha guardado correctamente en: {output_path}")


# 6. Notes

Colega hasta aqui se exporta un archivo en formato xlx, esta pendiente que sumerced agregue el paso de tratamiento para la columna de observaciones, la idea seria que lo numere y lo agregue aqui. por otra parte si quiere optimizar este codigo es libre de hacerlo, estoy abierto a cualquier sugerencia para mejorar. Gracias